In [ ]:
import pandas as pd
import numpy as np
!pip install pickle5
import pickle5 as pickle
!pip install cryptocmd
from cryptocmd import CmcScraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=262126 sha256=c2e0d6dd6f33905a451dc178a154a5d6df321074f39058d17024af19bde5bcc0
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 KB 2.9 MB/s eta 0:00:00


In [ ]:
Stock_name="BTC"

# initialise scraper without time interval for max historical data
scraper = CmcScraper(Stock_name)
# Pandas dataFrame for the same data
df = scraper.get_dataframe()
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')#.dt.strftime("%Y-%m-%d");
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df
data_for_valid_2=df.copy()

In [ ]:
data_for_valid_2.describe

In [ ]:
stock_data_2=stock_data[["Close",'Volume']]
stock_data_2.plot(subplots = True, figsize = (8, 8));

In [ ]:
Stock_name="BTC"
FUTURE_PERIOD_PREDICT =7  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "Close"

# initialise scraper without time interval for max historical data
scraper = CmcScraper(Stock_name)
# Pandas dataFrame for the same data
df = scraper.get_dataframe()
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')#.dt.strftime("%Y-%m-%d");
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df

df_date=df.copy()

df_date['lable_0']=(df_date['Close'].pct_change(periods=7)*100).shift(-7)

def vwap(dataframe, label='vwap', window=14, fillna=True):
        dataframe[label] = VolumeWeightedAveragePrice(high=dataframe['High'], low=dataframe['Low'], close=dataframe["Close"], volume=dataframe['Volume'], window=window, fillna=fillna).volume_weighted_average_price()
        return dataframe

d=vwap(df_date)

df_date['future'] = df_date[RATIO_TO_PREDICT].shift(-FUTURE_PERIOD_PREDICT)
def classify(current, future):
    if ((float(future) - float(current))/ float(current))>0.05:
        return 5
    if  0.01<=((float(future) - float(current))/ float(current))<0.05:
        return 4
    if  -0.01<=((float(future) - float(current))/ float(current))<0.01:
        return 3
    if  -0.05<=((float(future) - float(current))/ float(current))<-0.01:
        return 2
    if ((float(future) - float(current))/ float(current))<-0.05:
        return 1
df_date=df_date.dropna()
df_date['target'] = list(map(classify, df_date[RATIO_TO_PREDICT], df_date['future']))
print(df_date)

In [ ]:
times = sorted(main_df.index.values)  # get the times
last_10pct = sorted(main_df.index.values)[-int(0.1*len(times))]  # get the last 10% of the times
 
validation_main_df = main_df[(main_df.index >= last_10pct)]  # make the validation data where the index is in the last 10%
main_df = main_df[(main_df.index < last_10pct)]  # now the main_df is all the data up to the last 10%
 
main_df.head()
print(validation_main_df)

In [ ]:
main_df.to_excel("main_df.xlsx",sheet_name='main_df')
files.download('main_df.xlsx') 

In [ ]:
def preprocess_df(df):
 
  for co in df.columns:
    if co == "future":
      df = df.drop("future", 1)  # don't need this anymore.
    if co == "MID":
      df = df.drop("MID", 1)
 
  
  df1= df[['SHAKHES', 'VOL-SHAKHES','NUM-SHAKHES','VALUE-SHAKHES','seke','dollar','<OPEN>','<HIGH>','<LOW>','<CLOSE>','<VOL>','<VALUE>']]
  df2 = df1.pct_change()
  df2=df2.replace([np.inf, -np.inf], np.nan).dropna( how="all")
  df2.dropna(inplace=True)
 
  for col in df2.columns:
    df2[col] = preprocessing.scale(df2[col].values)  # scale between 0 and 1.
 
  df2.dropna(inplace=True)  # cleanup again... jic.
  df3=df.drop(['SHAKHES', 'VOL-SHAKHES','NUM-SHAKHES','VALUE-SHAKHES','seke','dollar','<OPEN>','<HIGH>','<LOW>','<CLOSE>','<VOL>','<VALUE>'], axis=1)
  #df3=df3.iloc[1:]
  for col in df3.columns:  # go through all of the columns
      if col != "target": # normalize all ... except for the target itself!
        df3[col] = preprocessing.scale(df3[col].values)
        #df3[col]=(df3[col]-df3[col].min())/(df3[col].max()-df3[col].min())
 
  df4=df2.merge(df3, right_index=True,left_index=True, how='left', sort=False)
 
  df5=df4[(np.abs(stats.zscore(df4)) < 3).all(axis=1)]
  df5.dropna(inplace=True)  # cleanup again... jic. # cleanup again... jic.
 
  sequential_data = []  # this is a list that will CONTAIN the sequences
  prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in
 
  for i in df4.values:  # iterate over the values
      prev_days.append([n for n in i[:-1]])  # store all but the target
      if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
          sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!
 
  random.shuffle(sequential_data)  # shuffle for good measure.
 
  buys = []  # list that will store our buy sequences and targets
  nothing=[] #kari nakon
  sells = []  # list that will store our sell sequences and targets
 
  for seq, target in sequential_data:  # iterate over the sequential data
      if target == 1 :  # if it's a "not buy"
          sells.append([seq, target])  # append to sells list
      if target == 2 or 3  :  # if it's a "not buy"
          nothing.append([seq, target])  # append to nothing list
      if target == 4 :  # otherwise if the target is a 1...
          buys.append([seq, target])  # it's a buy!
 
  random.shuffle(buys)  # shuffle the buys
  random.shuffle(nothing)
  random.shuffle(sells)  # shuffle the sells!
  
  lower = min(len(buys), len(sells),len(nothing))  # what's the shorter length?
  buys = buys[:lower]  # make sure both lists are only up to the shortest length.
  nothing = nothing[:lower]
  sells = sells[:lower]  # make sure both lists are only up to the shortest length.
 
  sequential_data = buys+sells+nothing#+nothing2 # add them together
  random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.
  
  X = []
  y = []
  
  for seq, target in sequential_data:  # going over our new sequential data
      X.append(seq)  # X is the sequences
      y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
 
  return np.array(X), y  # return X and y...and make X a numpy array!

In [ ]:
last_data=init_df_x.tail(SEQ_LEN)
last_data2=last_data.drop(['MID'], axis=1)

sequential_data = []  # this is a list that will CONTAIN the sequences
prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

for i in last_data2.values:  # iterate over the values
    prev_days.append([n for n in i[:-1]])  # store all but the target
    if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
        sequential_data.append([np.array(prev_days), i[-1]])

# Extract your training data
X_LAST_init = np.asarray(last_data2)
# Use hstack to and reshape to make the inputs a 3d vector
X_LSAT = np.hstack(X_LAST_init).reshape(1,SEQ_LEN,last_data2.shape[1])
#print(X_LSAT.shape)

In [ ]:
print(train_x.shape)
print(validation_x.shape)
print(X_LSAT.shape)

print(train_y)
print(validation_y)
print(last_data)

print(train_x.shape[2])
print(train_x[:,0,:])

In [ ]:
EPOCHS = 100  # how many passes through our data
BATCH_SIZE = 32  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.
 
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())
 
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(3, activation='softmax'))


opt = tf.keras.optimizers.Adam(lr=0.005, decay=1e-6)
 
# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])
#model.compile(optimizer='adam', loss='mse')
print(model.summary())

# **NY Stock Price Prediction RNN LSTM GRU**

In [12]:
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

#display parent directory and working directory
print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
print(os.getcwd()+':', os.listdir(os.getcwd()));

Instructions for updating:
non-resource variables are not supported in the long term


/: ['mnt', 'var', 'dev', 'etc', 'libx32', 'sys', 'opt', 'media', 'lib64', 'sbin', 'home', 'usr', 'boot', 'lib', 'tmp', 'srv', 'lib32', 'run', 'root', 'bin', 'proc', '.dockerenv', 'tools', 'datalab', 'content', 'NGC-DL-CONTAINER-LICENSE']
/content: ['.config', 'sample_data']


In [2]:
!pip install cryptocmd
from cryptocmd import CmcScraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 KB 666.0 kB/s eta 0:00:00


In [3]:
Stock_name="BTC"

# initialise scraper without time interval for max historical data
scraper = CmcScraper(Stock_name)
# Pandas dataFrame for the same data
df = scraper.get_dataframe()
'''
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')#.dt.strftime("%Y-%m-%d");
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df
'''

'\ndf[\'Date\'] = pd.to_datetime(df[\'Date\'],format=\'%Y-%m-%d\')#.dt.strftime("%Y-%m-%d");\ndf.set_index(\'Date\', inplace=True)\ndf.sort_index(inplace=True)\ndf\n'

In [4]:
df.tail()

,Date,Open,High,Low,Close,Volume,Market Cap
3566,2013-05-02,116.379997,125.599998,92.281898,105.209999,0.0,1.168517e+09
3567,2013-05-01,139.000000,139.889999,107.720001,116.989998,0.0,1.298955e+09
3568,2013-04-30,144.000000,146.929993,134.050003,139.000000,0.0,1.542813e+09
3569,2013-04-29,134.444000,147.488007,134.000000,144.539993,0.0,1.603769e+09
3570,2013-04-28,135.300003,135.979996,132.100006,134.210007,0.0,1.488567e+09


In [5]:
df.describe()

,Open,High,Low,Close,Volume,Market Cap
count,3571.000000,3571.000000,3571.000000,3571.000000,3.571000e+03,3.571000e+03
mean,11170.864698,11451.803912,10858.907721,11175.971744,1.408336e+10,2.065932e+11
std,15557.171038,15953.834416,15100.699888,15554.072133,1.928604e+10,2.939878e+11
min,68.504997,74.561096,65.526001,68.431000,0.000000e+00,7.784112e+08
25%,496.188995,509.938004,481.943512,495.868011,4.481485e+07,6.946328e+09
50%,5279.471142,5371.348516,5177.368514,5285.139008,4.396930e+09,9.327229e+10
75%,11904.367632,12164.688514,11639.725316,11916.517668,2.501136e+10,2.174276e+11
max,67549.735581,68789.625939,66382.061008,67566.830088,3.509679e+11,1.274831e+12


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        3571 non-null   datetime64[ns]
 1   Open        3571 non-null   float64       
 2   High        3571 non-null   float64       
 3   Low         3571 non-null   float64       
 4   Close       3571 non-null   float64       
 5   Volume      3571 non-null   float64       
 6   Market Cap  3571 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 195.4 KB


In [7]:
df.rename(columns={'Open':'open','High':'high','Low':'low', 'Close':'close','Volume':'volume'}, inplace=True)